In [10]:
import os
from PIL import Image

image_folder = "./image_face_detection"

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as fn
import torchvision.ops as ops
from net import Net


In [ ]:
face_detection_net = Net()
face_detection_net.load_state_dict(torch.load("D:\5IF\OT2_ML\deep_learning_project\saved_model.pt"))

In [9]:
x = [[1,2,3],[2,3,4]]
print(x[:1])

[[1, 2, 3]]


In [5]:
# initialize variables used for the object detection procedure
WIDTH = 600
PYR_SCALE = 1.5
WINDOW_STEP = 24
ROI_SIZE = (168,168)
INPUT_SIZE = (36,36)

In [ ]:
# Define the region of interest parameters
left = 0  # Define the left coordinate of the region
top = 0  # Define the top coordinate of the region
width = 36  # Define the width of the region
height = 36  # Define the height of the region

rois = []
locs = []

In [3]:
def sliding_window(image, step, ws):
	# slide a window across the image
	for y in range(0, image.shape[0] - ws[1], step):
		for x in range(0, image.shape[1] - ws[0], step):
			# yield the current window
			yield (x, y, image[y:y + ws[1], x:x + ws[0]])

In [4]:
def image_pyramid(image, scale=1.5, minSize=(168, 168)):
	# yield the original image
	yield image
	# keep looping over the image pyramid
	while True:
		# compute the dimensions of the next image in the pyramid
		w = int(image.shape[1] / scale)
		image = fn.resize(image, w)
		# if the resized image does not meet the supplied minimum
		# size, then stop constructing the pyramid
		if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
			break
		# yield the next image in the pyramid
		yield image

In [ ]:
transform = transforms.Compose([  
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((36,36)),
    transforms.PILToTensor()
])

image_path = "./image_face_detection/0000eda1171fe14e.jpg"

In [ ]:
#for filename in os.listdir(image_folder):
#    image_path = os.path.join(image_folder, filename)
img = Image.open(image_path)

W, H = img.size

pyramid = image_pyramid(img, scale=PYR_SCALE, minSize=ROI_SIZE)

for image in pyramid:
    # determine the scale factor between the *original* image
    # dimensions and the *current* layer of the pyramid
    scale = W / float(image.shape[1])
    # for each layer of the image pyramid, loop over the sliding
    # window locations
    for (x, y, roiOrig) in sliding_window(image, WINDOW_STEP, ROI_SIZE):
        # scale the (x, y)-coordinates of the ROI with respect to the
        # *original* image dimensions
        x = int(x * scale)
        y = int(y * scale)
        w = int(ROI_SIZE[0] * scale)
        h = int(ROI_SIZE[1] * scale)
        # take the ROI and preprocess it so we can later classify
        # the region using Keras/TensorFlow
        #roi = cv2.resize(roiOrig, INPUT_SIZE)
        #roi = img_to_array(roi)
        #roi = preprocess_input(roi)

        roi_tensor_gray = transform(roiOrig)

        # update our list of ROIs and associated coordinates
        rois.append(roi_tensor_gray)
        locs.append((x, y, x + w, y + h))


In [ ]:
# Create a TensorDataset
dataset = torch.utils.data.TensorDataset(*rois)

# Define batch size and other DataLoader parameters
#batch_size = 32

# Create a DataLoader
#dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
output = face_detection_net(dataset)

In [9]:
x = torch.tensor([3,4,5])
print(x.size(dim=0))

3


In [ ]:
probs = torch.nn.functional.softmax(output, dim=1)
probs_list = probs.tolist()

labels = {'valid_probs': [],
          'boxes': []}

In [ ]:
for i in range(0,len(probs_list)):
    if (probs_list[i][1] >= 0.95):
        box = locs[i]

        labels['valid_probs'].append(probs_list[i][1])
        labels['boxes'].append(box)


valid_box = ops.nms(torch.Tensor(labels['boxes']), torch.Tensor(labels['valid_probs']), iou_threshold=0.3)
        

In [22]:
image_path = "./image_face_detection/0000eda1171fe14e.jpg"
img = Image.open(image)

print(img)

img_tensor = transform(img) 
  
# print the converted Torch tensor 
print(img_tensor.shape) 

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x772 at 0x1B67D140C40>
torch.Size([1, 772, 1024])


In [ ]:
# Extract the region of interest
roi = img.crop((left, top, left + width, top + height))

# Resize the region to 36x36
#roi_resized = roi.resize((36, 36))

# Convert the image to grayscale
roi_gray = roi.convert('L')

rois.append(roi)
#locs.append((x, y, x + w, y + h))

In [ ]:
for image in pyramid:
	# determine the scale factor between the *original* image
	# dimensions and the *current* layer of the pyramid
	scale = W / float(image.shape[1])
	# for each layer of the image pyramid, loop over the sliding
	# window locations
	for (x, y, roiOrig) in sliding_window(image, WIN_STEP, ROI_SIZE):
		# scale the (x, y)-coordinates of the ROI with respect to the
		# *original* image dimensions
		x = int(x * scale)
		y = int(y * scale)
		w = int(ROI_SIZE[0] * scale)
		h = int(ROI_SIZE[1] * scale)
		# take the ROI and preprocess it so we can later classify
		# the region using Keras/TensorFlow
		roi = cv2.resize(roiOrig, INPUT_SIZE)
		roi = img_to_array(roi)
		roi = preprocess_input(roi)
		# update our list of ROIs and associated coordinates
		rois.append(roi)
		locs.append((x, y, x + w, y + h))